In [1]:
import os
import pandas as pd

In [2]:
%pwd

'd:\\software\\python_vs\\Bone_marrow_survival_prediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\software\\python_vs\\Bone_marrow_survival_prediction'

## Entity

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationconfig:
    root_dir : Path
    data_path : Path

## ConfigurationManager

In [6]:
from Bone_marrow_survival_prediction.constants import *
from Bone_marrow_survival_prediction.utils.comman import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
        ):
    
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
    
        create_directories([self.config.artifacts_root])
    
    def get_data_transformation_config(self) -> DataTransformationconfig:
        config = self.config.data_transformation
        
        create_directories([config.root_dir])
        
        data_transformation_config = DataTransformationconfig(
            root_dir = config.root_dir,
            data_path = config.data_path
        )
        
        return data_transformation_config
    

## Coponents

In [7]:
import os
from Bone_marrow_survival_prediction import logger
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import OrdinalEncoder
import pandas as pd
import numpy as np

In [12]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif
from imblearn.over_sampling import SMOTE
from collections import Counter
import logging

# Configure logger
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class DataTransformationConfig:
    # Add your configuration attributes here
    data_path: str
    root_dir: str
    # Define other configuration parameters as needed

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    @staticmethod
    def is_string_numeric(val):
        try:
            float(val)
            return True
        except ValueError:
            return False

    def preprocess_data(self):
        data = pd.read_csv(self.config.data_path)

        if "Disease" in data.columns and data["Disease"].dtype == "O":
            encoder = OrdinalEncoder()
            data["Disease_encoded"] = encoder.fit_transform(data[["Disease"]])
            data.drop("Disease", axis=1, inplace=True)

        data.replace("?", np.nan, inplace=True)
        data.dropna(inplace=True)

        for feature in data.columns:
            if data[feature].dtype == "O":
                is_numeric = data[feature].apply(self.is_string_numeric)
                if is_numeric.all():  # Apply conversion if all values are numeric
                    data[feature] = data[feature].astype("float64")
                    logger.info(f"The {feature} feature is converted into float")

        return data

    def train_test_split(self):
        data = self.preprocess_data()  # Preprocess data before splitting
        schema = {'target': 'TARGET_COLUMN_NAME'}  # Replace with your actual schema

        X = data.drop(columns=schema['target'])
        y = data[schema['target']]

        mi = mutual_info_classif(X, y)
        mi_df = pd.DataFrame({'Feature': X.columns, 'Mutual Information': mi})
        mi_df = mi_df.sort_values(by='Mutual Information', ascending=False)
        top_features = mi_df.head(5)['Feature']

        X = data[top_features]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        desired_percentage = 0.5

        current_counts = Counter(y_train)
        total_samples = len(y_train)
        minority_class = min(current_counts, key=current_counts.get)
        majority_class = max(current_counts, key=current_counts.get)

        desired_minority_count = int(total_samples * desired_percentage)
        minority_samples_needed = desired_minority_count - current_counts[minority_class]

        # Apply SMOTE
        smote = SMOTE(sampling_strategy={minority_class: current_counts[minority_class] + minority_samples_needed})
        X_train, y_train = smote.fit_resample(X_train, y_train)

        X_train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        y_train.to_csv(os.path.join(self.config.root_dir, "y_train.csv"), index=False)
        X_test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)
        y_test.to_csv(os.path.join(self.config.root_dir, "y_test.csv"), index=False)

        logger.info("Split data into train and test sets")

# Example usage
# config = DataTransformationConfig(data_path='path/to/data.csv', root_dir='path/to/output/')
# dt = DataTransformation(config)
# dt.train_test_split()


ModuleNotFoundError: No module named 'imblearn'

In [13]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(data_transformation_config)
    #data_transformation.preprocess_data()
    data_transformation.train_test_split()
except Exception as e:
    raise e
    
    

[ 2025-01-22 22:36:15,228: INFO: comman: yaml file: config\config.yaml loaded successfully]
[ 2025-01-22 22:36:15,228: INFO: comman: yaml file: params.yaml loaded successfully]
[ 2025-01-22 22:36:15,236: INFO: comman: yaml file: schema.yaml loaded successfully]
[ 2025-01-22 22:36:15,236: INFO: comman: created directory at: artifacts]
[ 2025-01-22 22:36:15,243: INFO: comman: created directory at: arifacts/data_validation]


ValueError: Expected a 2-dimensional container but got <class 'pandas.core.series.Series'> instead. Pass a DataFrame containing a single row (i.e. single sample) or a single column (i.e. single feature) instead.